# TMP

In [ ]:
def plot_basic_shared_branches(sim):
    plt.clf()
    A1, A2, shared = [], [], []

    for s in sim:
        tree = s.trees().next()
        p = tree.get_parent(0)
        if tree.get_parent(1) == p:
            A1.append(tree.get_branch_length(p))
        elif tree.get_parent(2) == p:
            A2.append(tree.get_branch_length(p))
        else:
            shared.append(tree.get_branch_length(tree.get_parent(1)))
    
    colors = ['b', 'r', 'g']
    plt.hist([A1, A2, shared], 20, color=colors)
    plt.show()
    
#     print(A1, A2, shared)

def plot_basic_shared_mutations():
    pass

def plot_count_branch_sharing(simulation=sim_basic_structure, reps=200, args=None):
    plt.clf()
    A1, A2 = [], []
    
    for _ in range(reps):
        a1, a2 = 0, 0
        for s in simulation(**args):
            tree = s.trees().next()
            p = tree.get_parent(0)
            if tree.get_parent(1) == p:
                a1 += 1
            elif tree.get_parent(2) == p:
                a2 += 1
            
        A1.append(a1)
        A2.append(a2)
    
    plt.scatter(A1, A2)
    
    test = sum([1 if t>0 else 0 for t in [i - j for i,j in zip(A1,A2)]])

    
    lower = min(A1 + A2) - 10
    upper = max(A1 + A2) + 50
    
    x = np.arange(lower, upper, 10)
    y = x    
    plt.plot(x,y, '--', color='red')
    plt.xlabel('A1 segment matches')
    plt.ylabel('A2 segment matches')
    plt.xlim(lower, upper)
    plt.ylim(lower, upper)
    
    plt.show()
    
    return test, len(A1) - test, len(A1)

In [ ]:
args = {'Ne':5e4, 
        'join_1':240, 
        'join_2':100,
        'merge':1000, 
        'ancestry_1':0.1,
        'scale_1':10, 
        'scale_2':2,
        'reps':5000,
        'mu': 1.2e-8,
        'length':10000}

def probability_segment_matches(args):
    lmb1, lmb2 = 1/args['scale_1'], 1/args['scale_2']
    t1, t2 = (args['merge'] - args['join_1'])/(2*args['Ne']), (args['merge'] - args['join_2'])/(2*args['Ne'])
    p0 = 0
    p1 = args['ancestry_1']
    p2 = 1 - p1 - p0
    
    theta = 4*args['Ne']*args['mu']*args['length']

    A = p1*exp(-t1/lmb1)/(lmb1 + 1)
    B = (p0 + p1*exp(-t1/lmb1) + p2*exp(-t2/lmb2))
    
    
    d1 = 1/lmb1 - theta/2
    d2 = 1 + theta/2
    
    I = p2*(1 - exp(-t2/lmb2)) + (2/3)*B
    I += A*(exp(d1*t1)-1)/d1 + A*(exp(-d2*t1)-1)/d2
    I += A*(exp((1 + 1/lmb1)*t1)-1)*(exp(-(1 + theta/2)*t1))/d2
    I += B/(3*d2)
    
    return 1-I  

prb = probability_segment_matches(args)
sim_bin = []
for i in range(1000):
    sim_bin.append(np.random.binomial(args['reps'], prb))

print(prb*args['reps'], '\n')

def sim_segment_matches(args):
    matches = []
    for s in sim_basic_structure(**args):
        tree = s.trees().next()
        p = tree.get_parent(0)
        if tree.get_parent(1) == p:
            segment_mutations = []
            for m in tree.mutations():
                if m.node == p:
                    segment_mutations.append(m)
            if segment_mutations:
                matches.append(segment_mutations)
    return len(matches)

y = []
for _ in range(1000):
#     print(sim_segment_matches(args)/args['reps'])
    y.append(sim_segment_matches(args))

print(np.mean(y))
plt.hist(y, bins=100, cumulative=True, normed=True, histtype='step', color='blue', linewidth=2)
plt.hist(sim_bin, bins=100, cumulative=True, normed=True, histtype='step', color='red', linewidth=2)
plt.show()

In [ ]:
def test_multinomial(chromosomes=1000, reps=args['reps'], o=0, t=0):
    out = []
    for _ in range(chromosomes):
        out.append(np.random.multinomial(reps, [o, t, 1 - o -t]))
    
    count = [0,0]
    for i in out:
        if i[0]>i[1]:
            count[0] += 1
        else:
            count[1] += 1
    
    return count, count[0]/(count[0]+count[1])

prob = test_multinomial(chromosomes = 100000, o=one[1], t=two[1])

print('Probability more A1 than A2 matches:', prob[1])

n = 8
p = prob[1]

# print(n,p,)

p_val = 0
for k in 0,1,2:
    bin_prob = scipy.special.comb(8, k)*(p**k)*((1-p)**(n-k))
    p_val += bin_prob
#     print(bin_prob)

print(n, p, p_val)

In [ ]:
args = {'Ne':1e4, 
        'join_1':0, 
        'join_2':0,
        'merge':2400, 
        'ancestry_1':1,
        'scale_1':1, 
        'scale_2':1,
        'reps':10000,
        'mu': 1.2e-8,
        'length':5000}

# one,two = compare_theoretical_pop_matches(args)
# rv = scipy.stats.multinomial(args['reps'], [one[1], two[1], 1 - one[1] - two[1]])
# test_1 = round(one[1]*args['reps'])
# test_2 = round(two[1]*args['reps'])
# test_N = args['reps'] - test_1 - test_2
# print('theoretical expected values:', one, two)

# x = np.arange(1000, 6000, 10)
# y = []
# for i in x:
#     args['merge'] = i
#     one,two = compare_theoretical_pop_matches(args)
#     rv = scipy.stats.multinomial(args['reps'], [one[1], two[1], 1 - one[1] - two[1]])
#     y.append(rv.pmf([test_1, test_2, test_N]))
    
# plt.plot(x,y)
# plt.show()    

def basic_pop_matches(simulation=sim_basic_structure, reps=10, args=args):
    plt.clf()
    A1, A2 = [], []
    
    for _ in range(reps):
        a1, a2 = 0, 0
        for s in simulation(**args):
            tree = s.trees().next()
            p = tree.get_parent(0)
            if tree.get_parent(1) == p:
                muts = []
                for m in tree.mutations():
                    if m.node == p:
                        muts.append(m)
                if muts:
                    a1 += 1
            elif tree.get_parent(2) == p:
                muts = []
                for m in tree.mutations():
                    if m.node == p:
                        muts.append(m)
                if muts:
                    a2 += 1
            
        yield a1, a2
        
args_tmp = args.copy()

lower = 1000
upper = 6000
mles = []
for sample in basic_pop_matches(reps=100):
#     print(sample, args['merge'])
    x = np.arange(lower, upper, 10)
    y = []
    for i in x:
        args_tmp['merge'] = i
        one,two = compare_theoretical_pop_matches(args_tmp)
        rv = scipy.stats.multinomial(args['reps'], [one[1], two[1], 1 - one[1] - two[1]])
        y.append(rv.pmf([sample[0], sample[1], args['reps'] - sample[0] - sample[1]]))
    
    plt.plot(x,y, alpha=0.4, color='blue', linewidth = 0.2)
    
    
    mles.append(x[y.index(max(y))])

plt.xlim(lower, upper)
plt.show()

plt.boxplot(mles, notch=True)
plt.show()

In [ ]:
args = {'Ne':1e4, 
        'join_1':0, 
        'join_2':0,
        'merge':2400, 
        'ancestry_1':1,
        'scale_1':1, 
        'scale_2':1,
        'reps':10000,
        'mu': 1.2e-8,
        'length':5000}

one,two = compare_theoretical_pop_matches(args)
print(compare_simulated_pop_matches(args=args, reps=100))
print('theoretical expected values:', one[1]*args['reps'], two[1]*args['reps'])

In [ ]:
args = {'Ne':1e4, 
        'join_1':240, 
        'join_2':100,
        'merge':1000, 
        'ancestry_1':0.9,
        'scale_1':3, 
        'scale_2':1,
        'reps':10000}

def prob_shared_branch_length(l, args):
    lmb1, lmb2 = 1/args['scale_1'], 1/args['scale_2']
    t1, t2 = (args['merge'] - args['join_1'])/(2*args['Ne']), (args['merge'] - args['join_2'])/(2*args['Ne'])
    p0 = 0
    p1 = args['ancestry_1']
    p2 = 1 - p1 - p0
    
    A = p1/(lmb1*(lmb1 + 1))
    B = (p0 + p1*(1-(1 - exp(-lmb1*t1))/lmb1) + p2*(1-(1 - exp(-lmb2*t2))/lmb2))
    F0 = p2*(1 - exp(-lmb2*t2))/lmb2 + (2/3)*B
    
    if l == 0:
        P = F0
    else:
        P = A*exp(-(lmb1*t1 + l))*(exp((lmb1+1)*min(l,t1) - 1)) + B*exp(-l)/3
    
    return P


def cumulative_shared_branch_length(l, args, pop):
    lmb1, lmb2 = 1/args['scale_1'], 1/args['scale_2']
    t1, t2 = (args['merge'] - args['join_1'])/(2*args['Ne']), (args['merge'] - args['join_2'])/(2*args['Ne'])
    p0 = 0
    p1 = args['ancestry_1']
    p2 = 1 - p1 - p0

    A = p1*(exp(-t1/lmb1)/(lmb1 + 1))
    B = (p0 + p1*exp(-t1/lmb1) + p2*exp(-t2/lmb2))
    F0 = p2*(1 - exp(-t2/lmb2)) + (2/3)*B
    
    if l == 0:
        return F0
    elif l < t1:
        return F0 + A*(lmb1*exp(l/lmb1) + exp(-l) -lmb1 -1) + (B/3)*(1 - exp(-l))
    else:
        Ft1 = F0 + A*(lmb1*exp(t1/lmb1) + exp(-t1) -lmb1 -1) + (B/3)*(1 - exp(-t1))
        return Ft1 + (A*(exp((1 + 1/lmb1)*t1)-1)+B/3)*(exp(-t1) - exp(-l))

x = np.arange(0, 4, 0.01)
y = [cumulative_shared_branch_length(i, args, 1) for i in x]
plt.plot(x,y)
        
L = []
L0 = []
for s in sim_basic_structure(**args):
    tree = s.trees().next()
    p = tree.get_parent(0)
    if tree.get_parent(1) == p:
        l = tree.get_branch_length(p)/(2*args['Ne'])
        L.append(l)
    else:
        L0.append(0)

plt.hist(L+L0, bins=500, normed=True, cumulative=True, histtype='step', color='r', linewidth=2.0)
plt.xlim(0,4)
plt.ylim(0.5,1)
plt.show()
print(len(L0)/args['reps'], cumulative_shared_branch_length(0, args, 1))
# print(args)

In [ ]:
def compare_theoretical_pop_matches(args):
   
    lmb1, lmb2 = 1/args['scale_1'], 1/args['scale_2']
    
    t1, t2 = (args['merge'] - args['join_1'])/(2*args['Ne']), (args['merge'] - args['join_2'])/(2*args['Ne'])
    if t1<0 or t2<0:
        raise('Invalid join times')
    
    p1 = args['ancestry_1']
    p2 = args['ancestry_2']
    if p1 + p2 > 1:
        raise Exception('Theoretical ancestry proportions exceed 1')
    else:
        p0 = 1 - p1 - p2
    
    theta = 4*args['Ne']*args['mu']*args['length']
    B = (p0 + p1*exp(-t1/lmb1) + p2*exp(-t2/lmb2))
    d2 = 1 + theta/2
    
    A = {1:p1*exp(-t1/lmb1)/(lmb1 + 1),
         2:p2*exp(-t2/lmb2)/(lmb2 + 1)}
    d1 = {1: 1/lmb1 - theta/2,
          2: 1/lmb2 - theta/2}   
    t = {1: t1,
         2: t2}
    I0 = {1: p2*(1 - exp(-t2/lmb2)) + (2/3)*B,
          2: p1*(1 - exp(-t1/lmb1)) + (2/3)*B}
    lmb = {1:lmb1,
           2:lmb2}
    
    out = []
    for i in 1,2:
        I = I0[i]
        I += A[i]*(exp(d1[i]*t[i])-1)/d1[i] + A[i]*(exp(-d2*t[i])-1)/d2
        I += A[i]*(exp((1 + 1/lmb[i])*t[i])-1)*(exp(-(1 + theta/2)*t[i]))/d2
        I += B/(3*d2)
        out.append(1-I)
    return out